In [1]:
import sys
sys.path.insert(1, '/home/ubuntu/QA_code/QA_application/training/Non_Distributed')
import torch
import torch.nn as nn
import timm
import wandb
import numpy as np
import os
import random
from import_packages.dataset_partition import split_equal_into_val, split_equal_into_test # noqa
from import_packages.dataset_class import Dataset
from import_packages.train_val_to_ids import train_val_to_ids
from import_packages.checkpoint import save_checkpoint

In [3]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
sys.path.insert(1, '/home/ubuntu/QA_code/QA_application/training/Non_Distributed')
import torch
import torch.nn as nn
import timm
import wandb
import numpy as np
import os
import random
from import_packages.dataset_partition import split_equal_into_val, split_equal_into_test # noqa
from import_packages.dataset_class import Dataset
from import_packages.train_val_to_ids import train_val_to_ids
from import_packages.checkpoint import save_checkpoint

In [3]:
# Deterministic Behavior
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
np.random.seed(seed)
random.seed(seed)
# CuDA Determinism
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [5]:
batch_size = 8
temp_train,temp_valid= split_equal_into_val(csv_file='/home/ubuntu/QA_code/QA_application/Processed_Input_files/Split_folders/2_cases_train_val_20220727-181159.csv', stratify_colname='labels',no_of_classes=2) # noqa
temp_test = split_equal_into_test(csv_file='/home/ubuntu/QA_code/QA_application/Processed_Input_files/Split_folders/2_cases_test_20220727-181159.csv', stratify_colname='labels') # noqa
partition, labels=train_val_to_ids(temp_train, temp_valid, temp_test, stratify_columns='labels') # noqa
training_set = Dataset(partition['train_set'], labels, root_dir='/home/ubuntu/EyePacs_Lenke_Dataset_Division_64/train', train_transform=True) # noqa
validation_set = Dataset(partition['val_set'],labels,root_dir='/home/ubuntu/EyePacs_Lenke_Dataset_Division_64/train',valid_transform = True) # noqa
# test_set = Dataset(partition['test_set'],labels,root_dir='/home/ubuntu/EyePacs_Lenke_Dataset_Division_64/test',test_transform=True) # noqa
train_loader = torch.utils.data.DataLoader(training_set, shuffle=True, pin_memory=True, num_workers=32, batch_size=batch_size) # noqa
val_loader = torch.utils.data.DataLoader(validation_set,shuffle=True, pin_memory=True, num_workers=32, batch_size=batch_size) # noqa

In [6]:
data_transfer = {'train': train_loader,
                 'valid': train_loader
                #  'test': test_loader
                 }

In [7]:
model_transfer = timm.create_model('convnext_tiny_in22k', pretrained=True,num_classes=2) # noqa


In [8]:
model_transfer1 = timm.create_model('resnet50d', pretrained=True,num_classes=2);model_transfer1

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [9]:
lr = 3e-4
optimizer = torch.optim.SGD(model_transfer.parameters(), lr=lr)
optimizer.zero_grad()
criterion_transfer = nn.BCEWithLogitsLoss()

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_transfer.to(device)
criterion_transfer.to(device)
# %%
if torch.cuda.device_count() > 1:
    model_transfer = nn.DataParallel(model_transfer)
    criterion_transfer = criterion_transfer
model_transfer.to(device)

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU(approximate=none)
            (drop1): Dropout(p=0.0, inplace=False)
            (fc2): Linear(in_features=384, out_features=96, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, el

In [10]:

valid_loss_min = np.Inf
for epoch in range(1, 2):
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0.0
    total = 0.0
    accuracy = 0.0
    model_transfer.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.to('cuda', non_blocking=True), target.to('cuda', non_blocking = True) # noqa
        optimizer.zero_grad()
        output = model_transfer(data)
        print("Shape of output:",output.shape)
        print("Shape of target:",target.shape)
        pred = torch.max(output,dim=1, keepdim=True)[0]
        # print("Shape of output:",pred.shape)
        print("Target - ",target)
        print("Output - ",output)
        target = torch.unsqueeze(target,dim=1)
        print("Shape of output:",output.shape)
        print("Shape of output:",target.shape)
        # target = target.to(torch.double)
        # pred = pred.to(torch.double)
        loss = criterion_transfer(pred.float(), target.float())
        pred = torch.argmax(output, dim=1)
        print("Pred", pred)
        break
        

Shape of output: torch.Size([8, 2])
Shape of target: torch.Size([8])
Target -  tensor([0, 0, 0, 0, 0, 0, 0, 0])
Output -  tensor([[ 0.5951,  0.1505],
        [ 0.2226,  0.2643],
        [-0.0270,  0.3440],
        [ 0.4433,  0.2618],
        [ 0.1023,  0.0470],
        [ 0.1772, -0.1040],
        [ 0.0660,  0.1165],
        [ 0.5347,  0.4978]], grad_fn=<AddmmBackward0>)
Shape of output: torch.Size([8, 2])
Shape of output: torch.Size([8, 1])
Pred tensor([0, 1, 1, 0, 0, 0, 1, 0])


In [14]:
 loss = nn.BCEWithLogitsLoss()
 input = torch.randn(3, requires_grad=True)
 target = torch.empty(3).random_(2)


In [26]:
input.shape

torch.Size([3])

In [29]:
target.shape

torch.Size([1])

In [15]:
output = loss(input, target)

In [36]:
model_transfer.eval()
for batch_idx, (data, target) in enumerate(train_loader):
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    output = model_transfer(data).squeeze()
    print("Output-", output)
    print("Shape of Output -", output.shape)
    print("Target-",target)
    pred = torch.max(output, dim=1, keepdim=True)[1]
    print("Pred-", pred)
    correct = pred.eq(target.data.view_as(pred)).numpy()
    corrects = np.sum(pred.eq(target.data.view_as(pred)).numpy())
    print(correct)
    print(corrects)
    print(data.size(0))
    break


Output- tensor([[ 0.1684,  0.4039],
        [ 0.2945, -0.0676],
        [ 0.1014, -0.1686],
        [-0.0722, -0.2214],
        [ 0.2072,  0.1229],
        [ 0.0465, -0.0711],
        [ 0.2358, -0.0007],
        [ 0.2484,  0.2329]], grad_fn=<SqueezeBackward0>)
Shape of Output - torch.Size([8, 2])
Target- tensor([0, 0, 0, 0, 0, 0, 0, 0])
Pred- tensor([[1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])
[[False]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]
7
8


In [22]:
a = torch.randn(1, 3)
a

tensor([[ 0.2362,  1.1819, -0.5951]])

In [37]:
torch.max(a,keepdim=True,dim=1)

torch.return_types.max(
values=tensor([[1.1819]]),
indices=tensor([[1]]))

In [ ]:
trainAccuracy = (predicted == labels).float().sum()
trainAccuracy = 100 * trainAccuracy / labels.size(0)

In [ ]:
valid_loss_min = np.Inf
model_transfer.eval()
for epoch in range(1, 2):
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0.0
    total = 0.0
    accuracy = 0.0
    model_transfer.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.to('cuda', non_blocking=True,dtype), target.to('cuda', non_blocking = True) # noqa
        optimizer.zero_grad()
        output = model_transfer(data)
        print("Shape of output:",output.shape)
        print("Shape of target:",target.shape)
        # # # output = torch.squeeze(output)
        # # print("Shape of output:",output.shape)
        # print("Target - ",target)
        # print("Output - ",output)
        print("Shape of target:",target.shape)
        print("Target - ",target)
        print("Output - ",output)
        pred = torch.max(output, dim=1, keepdim=True)
        loss = criterion_transfer(pred, target)
        # print(loss)
        break

In [39]:

valid_loss_min = np.Inf
for epoch in range(1, 2):
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0.0
    total = 0.0
    accuracy = 0.0
    model_transfer.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.to('cuda', non_blocking=True,dtype), target.to('cuda', non_blocking = True) # noqa
        optimizer.zero_grad()
        output = model_transfer(data)
        print("Shape of output:",output.shape)
        print("Shape of target:",target.shape)
        # # # output = torch.squeeze(output)
        # # print("Shape of output:",output.shape)
        # print("Target - ",target)
        # print("Output - ",output)
        print("Shape of target:",target.shape)
        print("Target - ",target)
        print("Output - ",output)
        loss = criterion_transfer(pred, target)
        # print(loss)
        break

Shape of output: torch.Size([1, 2])
Shape of target: torch.Size([1])
Shape of target: torch.Size([1])
Target -  tensor([0], device='cuda:0')
Output -  tensor([[0.3863, 0.1197]], device='cuda:0', grad_fn=<GatherBackward>)


RuntimeError: result type Float can't be cast to the desired output type Long